In [10]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"

from sentry.runner import configure

configure()

from sentence_transformers import SentenceTransformer

from sentry.event_manager import get_event_type
from sentry.eventstore.models import Event
from sentry.grouping.grouping_info import get_grouping_info
from sentry.seer.similarity.utils import get_stacktrace_string
from sentry.utils import json
from sentry.grouping.grouping_info import get_grouping_info_from_variants

model = SentenceTransformer("jinaai/jina-embeddings-v2-base-code", trust_remote_code=True)
init = model.encode('i')


/Users/jodi/sentry/src/sentry/runner/initializer.py:199: DeprecatedSettingWarning: The EMAIL_HOST setting is deprecated. Please use SENTRY_OPTIONS['mail.host'] instead.
  warnings.warn(DeprecatedSettingWarning(options_mapper[k], "SENTRY_OPTIONS['%s']" % k))
/Users/jodi/sentry/src/sentry/runner/initializer.py:199: DeprecatedSettingWarning: The EMAIL_PORT setting is deprecated. Please use SENTRY_OPTIONS['mail.port'] instead.
  warnings.warn(DeprecatedSettingWarning(options_mapper[k], "SENTRY_OPTIONS['%s']" % k))
/Users/jodi/sentry/src/sentry/runner/initializer.py:199: DeprecatedSettingWarning: The EMAIL_HOST_USER setting is deprecated. Please use SENTRY_OPTIONS['mail.username'] instead.
  warnings.warn(DeprecatedSettingWarning(options_mapper[k], "SENTRY_OPTIONS['%s']" % k))
/Users/jodi/sentry/src/sentry/runner/initializer.py:199: DeprecatedSettingWarning: The EMAIL_HOST_PASSWORD setting is deprecated. Please use SENTRY_OPTIONS['mail.password'] instead.
  warnings.warn(DeprecatedSettingWa

Exception: Cannot specify both SENTRY_OPTIONS['redis.clusters'] option and SENTRY_REDIS_OPTIONS setting.

In [ ]:
def prettify_grouping_output(output):
    prettified = ""
    for item in output:
        name, level = item
        prettified += " " * level + name + "\n"
    return prettified

def recurse_grouping_variant(ret, component, level):
    if component['contributes'] and component['name']:
        ret.append([component['name'], level])
    if component['values']:
        for value in component['values']:
            if isinstance(value, dict) and component['contributes']:
                recurse_grouping_variant(ret, value, level+1)
            else:
                if component['contributes'] and value:
                    ret.append([value, level])
    
    if level == 0:
        return ret

def debug_grouping_variant(variant):
    ret = []
    recurse_grouping_variant(ret, variant['component'], 0)
    return prettify_grouping_output(ret)



def get_seer_input(event):
    grouping_info = get_grouping_info(None, project=1, event=event)
    stacktrace_string = get_stacktrace_string(grouping_info)
    return stacktrace_string


def get_event(path):
    with open(path, "r") as f:
        event_json = json.load(f)

    event = Event(
        event_id=event_json["event_id"],
        project_id=1,
        group_id=None,
    )
    event.data = event_json
    return event


event1 = get_event("event1.json")
event2 = get_event("event2.json")

seer_input1 = get_seer_input(event1)
seer_input2 = get_seer_input(event2)

e1 = model.encode(seer_input1)
e2 = model.encode(seer_input2)
event1_group_info = get_grouping_info_from_variants(event1.get_grouping_variants())['app']
event2_group_info = get_grouping_info_from_variants(event2.get_grouping_variants())['app']






In [ ]:
print(f"event1 hash: {event1.get_primary_hash()}")
print(f"event2 hash: {event2.get_primary_hash()}")
print()
print("seer_input event1")
print()
print(seer_input1)
print()
print("seer_input event2")
print()
print(seer_input2)
print()
print('similarity:',cosine_similarity(np.array(e1).reshape(1, -1), np.array(e2).reshape(1, -1))[0][0])
print()
print('info used for grouping:')
print()
print('event1:')
print(debug_grouping_variant(event1_group_info))
print()
print('event2:')
print(debug_grouping_variant(event2_group_info))


In [17]:
from sentry.seer.similarity.utils import get_stacktrace_string
s = {"system":{"key":"system","type":"component","description":"exception stack-trace","hash":"0398a1bb8c73828ef2f51c9fa1068d2d","component":{"id":"system","name":None,"contributes":True,"hint":None,"values":[{"id":"exception","name":"exception","contributes":True,"hint":None,"values":[{"id":"stacktrace","name":"stack-trace","contributes":True,"hint":None,"values":[{"id":"frame","name":None,"contributes":False,"hint":"ignored by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["_doRecord"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["let ret = _applyRecorderSegment({ segment, ctx: this, args, segDesc, shim, fn, name })"]}]},{"id":"frame","name":None,"contributes":False,"hint":"ignored by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["_applyRecorderSegment"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["ret = shim.applySegment(fn, segment, True, ctx, args, segDesc.inContext)"]}]},{"id":"frame","name":None,"contributes":False,"hint":"ignored by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["applySegment"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return contextManager.runInContext(segment, function runInContextCb() {"]}]},{"id":"frame","name":None,"contributes":False,"hint":"ignored by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.context-manager:async-local-context-manager"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["async-local-context-manager.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["runInContext"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return this._asyncLocalStorage.run(context, toInvoke)"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["node:async_hooks"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["node:async_hooks"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["run"]},{"id":"context-line","name":None,"contributes":None,"hint":None,"values":[]}]},{"id":"frame","name":None,"contributes":False,"hint":"ignored by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["runInContextCb"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return fnApply.call(func, context, args)"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["services:ErrorService"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["errorservice.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["errorHandler"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["Sentry.withScope(function (scope) {"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["@sentry.core.cjs:exports"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["exports.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["withScope"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return hub$1.withScope(rest[0]);"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["@sentry.core.cjs:hub"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["hub.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["withScope"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["maybePromiseResult = callback(scope);"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["services:ErrorService"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["errorservice.js"]},{"id":"function","name":None,"contributes":False,"hint":"ignored unknown function name","values":["<anonymous>"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["e = new ApiErrorService(error.code, error.message, error.type, 'info');"]}]}]},{"id":"type","name":None,"contributes":True,"hint":None,"values":["Api info 404 The User 2db296ed-71da-4fe6-8404-df1db4704c79 does not exist!"]},{"id":"value","name":None,"contributes":False,"hint":"ignored because stacktrace takes precedence","values":["The User <uuid> does not exist!"]}]}]},"config":{"id":"newstyle:2023-01-11","base":"newstyle:2019-10-29","strategies":["chained-exception:v1","csp:v1","expect-ct:v1","expect-staple:v1","hpkp:v1","message:v1","stacktrace:v1","template:v1","threads:v1"],"changelog":"* Added new language/platform specific stack trace grouping enhancements rules\n  that should make the default grouping experience better.\n  This includes JavaScript, Python, PHP, Go, Java and Kotlin.\n* Added ChukloadErrors via new built-in fingerprinting support.","delegates":["frame:v1","single-exception:v1","stacktrace:v1"],"hidden":False,"risk":1,"latest":True},"hashMismatch":False},"app":{"key":"app","type":"component","description":"in-app exception stack-trace","hash":"db1146368e57d26cc2a8fe65ad65fd21","component":{"id":"app","name":"in-app","contributes":True,"hint":None,"values":[{"id":"exception","name":"exception","contributes":True,"hint":None,"values":[{"id":"stacktrace","name":"stack-trace","contributes":True,"hint":None,"values":[{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["_doRecord"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["let ret = _applyRecorderSegment({ segment, ctx: this, args, segDesc, shim, fn, name })"]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["_applyRecorderSegment"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["ret = shim.applySegment(fn, segment, True, ctx, args, segDesc.inContext)"]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["applySegment"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return contextManager.runInContext(segment, function runInContextCb() {"]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.context-manager:async-local-context-manager"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["async-local-context-manager.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["runInContext"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return this._asyncLocalStorage.run(context, toInvoke)"]}]},{"id":"frame","name":None,"contributes":False,"hint":"non app frame","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["node:async_hooks"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["node:async_hooks"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["run"]},{"id":"context-line","name":None,"contributes":None,"hint":None,"values":[]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/*newrelic*/** -app -group)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["newrelic.lib.shim:shim"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["shim.js"]},{"id":"function","name":None,"contributes":True,"hint":None,"values":["runInContextCb"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return fnApply.call(func, context, args)"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["services:ErrorService"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["errorservice.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["errorHandler"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["Sentry.withScope(function (scope) {"]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/node_modules/** -app)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["@sentry.core.cjs:exports"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["exports.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["withScope"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["return hub$1.withScope(rest[0]);"]}]},{"id":"frame","name":None,"contributes":False,"hint":"marked out of app by stack trace rule (path:**/node_modules/** -app)","values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["@sentry.core.cjs:hub"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["hub.js"]},{"id":"function","name":None,"contributes":True,"hint":"trimmed javascript function","values":["withScope"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["maybePromiseResult = callback(scope);"]}]},{"id":"frame","name":None,"contributes":True,"hint":None,"values":[{"id":"module","name":None,"contributes":True,"hint":None,"values":["services:ErrorService"]},{"id":"filename","name":None,"contributes":False,"hint":"module takes precedence","values":["errorservice.js"]},{"id":"function","name":None,"contributes":False,"hint":"ignored unknown function name","values":["<anonymous>"]},{"id":"context-line","name":None,"contributes":True,"hint":None,"values":["e = new ApiErrorService(error.code, error.message, error.type, 'info');"]}]}]},{"id":"type","name":None,"contributes":True,"hint":None,"values":["Api info 404 The User 2db296ed-71da-4fe6-8404-df1db4704c79 does not exist!"]},{"id":"value","name":None,"contributes":False,"hint":"ignored because stacktrace takes precedence","values":["The User <uuid> does not exist!"]}]}]},"config":{"id":"newstyle:2023-01-11","base":"newstyle:2019-10-29","strategies":["chained-exception:v1","csp:v1","expect-ct:v1","expect-staple:v1","hpkp:v1","message:v1","stacktrace:v1","template:v1","threads:v1"],"changelog":"* Added new language/platform specific stack trace grouping enhancements rules\n  that should make the default grouping experience better.\n  This includes JavaScript, Python, PHP, Go, Java and Kotlin.\n* Added ChukloadErrors via new built-in fingerprinting support.","delegates":["frame:v1","single-exception:v1","stacktrace:v1"],"hidden":False,"risk":1,"latest":True},"hashMismatch":False}}
stacktrace_string = get_stacktrace_string(s)
print(stacktrace_string)


Api info 404 The User 2db296ed-71da-4fe6-8404-df1db4704c79 does not exist!: The User <uuid> does not exist!
  File "errorservice.js", function errorHandler
    Sentry.withScope(function (scope) {
  File "errorservice.js", function <anonymous>
    e = new ApiErrorService(error.code, error.message, error.type, 'info');
